<a href="https://colab.research.google.com/github/romualdojonathan7/application_test/blob/main/monitoramento_agendas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Monitoramento de Agendas**


*Objetivos*:
1.   agenda contratada x agenda ofertada;

In [1]:
import sys
import psycopg2
import pandas as pd
import datetime as dt
from datetime import timedelta as td
import numpy as np

* connect(database): Conexão com o banco de dados
* connect_dbTable(tableName, dbConnection, dtype=None): Selecionar uma tabela
* dict_replace(array, df_dict):replace a dictionary in a column dataframe

In [2]:
def connect(database):
    """ Connect to the PostgreSQL database server """
    connector = None

    try:
        # read connection parameters
        #parameters = config()

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        connector = psycopg2.connect(
            database=database,
            host="sias-campinas-producao.c9yiqiksj92l.sa-east-1.rds.amazonaws.com",
            user="powerbi",
            password="3A2qeYWXkjBOfWAuUdKH",
            port="5432"
        )

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 

    print("Connection successful")
    return connector

def connect_dbTable(tableName, dbConnection, dtype=None):
    
    # Testando a conexão
    df_sql = f""" select * from "{tableName}" """
    df = pd.read_sql(df_sql, dbConnection);
    
    pd.set_option('display.expand_frame_repr', False);
    
    if pd.notnull(dtype):
        for i in dtype:
            if i in df.columns:
                df[i] = df[i].astype(dtype[i])

    return(df)

def dict_replace(array, dfDict, dictPosition=None):
    new_array = array
    for key, value in dfDict.items():
        # print(key)
        # print(value)
        if pd.notnull(dictPosition):
          # print(value[dictPosition])
          new_array = new_array.replace(key, value[dictPosition])
        else:
          new_array = new_array.replace(key, value)
    print(new_array)
    return new_array

convert_to_hours = lambda x: '{:02.0f}:{:02.0f}'.format(x, (x % 1) * 60)

## > Conexão com a base e seleção das tabelas

In [3]:
# Conectando ao banco de dados
dbConnection = connect("sias")

Connecting to the PostgreSQL database...
Connection successful


In [5]:
# Selecionando a tabela que iremos trabalhar
schedules = connect_dbTable("schedules", dbConnection, dtype={'schedule_id': str, 'doctor_id': str, 'register_by_id': str, 'uta_id': str, 'specialties_id': str, 'service_list_id': str, 'care_line_list_id': str, })
calendar = connect_dbTable("calendar", dbConnection, dtype={'specialties_id': str})

In [6]:
dbConnection.close();
print('Database connection closed.')

Database connection closed.


### 1. Agenda contratada X agenda ofertada


*   Comparativo entre a carga horária contrata e a carga horária efetivamente disponibilizada



In [12]:
# AGENDA CONTRATADA ------------------------------------------------------------

# Transformando os tipos para datetime (caso contrário a operação de subtração não reconheceria os campos)
calendar['start'] = pd.to_datetime(calendar['start'])
calendar['end'] = pd.to_datetime(calendar['end'])

# Fazendo a diferença entre 'end' e 'start'
calendar['duration'] = calendar['end'].sub(calendar['start'], axis=0)

# Deixando a informação em um padrão que o powerbi entenda
calendar['duration_formatted'] = calendar['duration'].apply(lambda x: '{:0d}.{:02d}:{:02d}:{:02d}'.format(int(x.total_seconds() // 86400), int(x.total_seconds() % 86400 // 3600), int(x.total_seconds() % 3600 // 60), int(x.total_seconds() % 60)))

### 3. Detalhamento das agendas publicadas


*   Período de vigência;
*   Faixa etária (data de hoje - data de nascimento - classificado);
*   Procedimento;
*   Operador que publicou;
*   Quantidade de vagas (colunas dos dias*quantidade de recorrências).


In [ ]:
# columns = ['sun', 'mon', 'tue', 'wed', 'thu', 'fri', 'sat']

# df_schedules['Agendamentos/semana'] = df_schedules[columns].apply(lambda row: (row == True).sum(), axis=1)
# df_schedules['quantidade_vagas'] = df_schedules['quantidade_vagas'] * df_schedules['quantidade_recorrencias_dias']

In [14]:
# 2. Calcular quantos agendamentos tiveram por semana
columns = ['sun', 'mon', 'tue', 'wed', 'thu', 'fri', 'sat']
schedules['Agendamentos/semana'] = schedules.apply(lambda row: (row[columns] == True).sum() if row['recurring'] != False else 1, axis=1)

# 3. Calcular a hora inicial
schedules['duration_formatted'] = (schedules['duration'] / 60).abs().apply(convert_to_hours)

# 5. Criar coluna "atual" para saber os agendamentos a partir de hoje.
schedules['atual2'] = schedules.apply(lambda row: ('1' if row['recurringUntil'] >= dt.datetime.now().date() else '0') if row['recurringUntil'] != None else 1, axis=1)

# 6. Qtde de dias disponibilizados
schedules['quantidade_dias_disponibilizados'] = (schedules['recurringUntil'] - schedules['startingDate']).dt.days
schedules['quantidade_dias_disponibilizados'] = schedules['quantidade_dias_disponibilizados'].apply(lambda row: row if row != 0 else 1)

# 7. Carga disponibilizada em minutos
schedules['dias_total'] = schedules.apply(lambda row: row['quantidade_dias_disponibilizados'] if row['recurring'] != 0 or row['recurringUntil'] != None else 1, axis=1)
schedules['carga_minutos'] = schedules[['duration', 'dias_total', 'Agendamentos/semana']].prod(axis=1).abs()

# 8.
schedules['Agendamentos/semana total'] = schedules['Agendamentos/semana'] * (schedules['dias_total'] / 7)

## FINAL

In [ ]:
# Conectando ao banco de dados
dbConnection = connect("sias")

# Selecionando a tabela que iremos trabalhar
schedules = connect_dbTable("schedules", dbConnection)
calendar = connect_dbTable("calendar", dbConnection)

dbConnection.close();
print('Database connection closed.')

# AGENDA CONTRATADA ------------------------------------------------------------

# Transformando os tipos para datetime (caso contrário a operação de subtração não reconheceria os campos)
calendar['start'] = pd.to_datetime(calendar['start'])
calendar['end'] = pd.to_datetime(calendar['end'])

# Fazendo a diferença entre 'end' e 'start'
calendar['duration'] = calendar['end'].sub(calendar['start'], axis=0)

# Deixando a informação em um padrão que o powerbi entenda
calendar['duration_formatted'] = calendar['duration'].apply(lambda x: '{:03d}:{:02d}:{:02d}:{:02d}'.format(int(x.total_seconds() // 86400), int(x.total_seconds() % 86400 // 3600), int(x.total_seconds() % 3600 // 60), int(x.total_seconds() % 60)))

# ----------------------------------------------------------------------------------------
# 2. Calcular quantos agendamentos tiveram por semana
columns = ['sun', 'mon', 'tue', 'wed', 'thu', 'fri', 'sat']
schedules['Agendamentos/semana'] = schedules.apply(lambda row: (row[columns] == True).sum() if row['recurring'] != False else 1, axis=1)

# 3. Calcular a hora inicial
schedules['duration_formatted'] = (schedules['duration'] / 60).abs().apply(convert_to_hours)

# 5. Criar coluna "atual" para saber os agendamentos a partir de hoje.
schedules['atual2'] = schedules.apply(lambda row: ('1' if row['recurringUntil'] >= dt.datetime.now().date() else '0') if row['recurringUntil'] != None else 1, axis=1)

# 6. Qtde de dias disponibilizados
schedules['quantidade_dias_disponibilizados'] = (schedules['recurringUntil'] - schedules['startingDate']).dt.days
schedules['quantidade_dias_disponibilizados'] = schedules['quantidade_dias_disponibilizados'].apply(lambda row: row if row != 0 else 1)

# 7. Carga disponibilizada em minutos
schedules['dias_total'] = schedules.apply(lambda row: row['quantidade_dias_disponibilizados'] if row['recurring'] != 0 or row['recurringUntil'] != None else 1, axis=1)
schedules['carga_minutos'] = schedules[['duration', 'dias_total', 'Agendamentos/semana']].prod(axis=1).abs()

# 8.
schedules['Agendamentos/semana total'] = schedules['Agendamentos/semana'] * (schedules['dias_total'] / 7)